In [ ]:
%pip install yt-dlp
%pip install colorama
!git clone https://github.com/inmare/data-extractor.git

In [ ]:
from playlist_generator import playlist

# 아래의 큰 따옴표 안에 들어있는 문자를 파일 이름으로 바꾸고 프로그램을 실행하세요.
excel_file_name = "지우고 수정해야 될 부분"
playlist.generate(excel_file_name)